In [1]:
import pandas as pd
import requests
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import sklearn

In [2]:
# sms_messages spam dection by text analysis

In [3]:
sms_messages =pd.read_table('SMSSpamCollection.tsv',header=None,names=['label','messages'])

In [4]:
sms_messages.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
sms_messages.describe()

,label,messages
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
sms_messages.tail()

,label,messages
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [7]:
sms_messages.shape

(5572, 2)

In [8]:
sms_messages.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
sms_messages['label_number'] = sms_messages.label.map({'ham':0,'spam':1})

In [10]:
sms_messages.head()

,label,messages,label_number
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
x=sms_messages.messages
y=sms_messages.label_number

In [12]:
print(x.shape,y.shape)

(5572,) (5572,)


In [13]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=13)

In [14]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(3900,) (3900,) (1672,) (1672,)


### text Analysis start

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
vector = CountVectorizer()

In [17]:
vector.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [18]:
x_train_dtmatrix = vector.transform(x_train)

In [19]:
x_train_dtmatrix

<3900x7125 sparse matrix of type '<class 'numpy.int64'>'
	with 51886 stored elements in Compressed Sparse Row format>

In [20]:
x_test_dtmatrix = vector.transform(x_test)

In [21]:
x_test_dtmatrix

<1672x7125 sparse matrix of type '<class 'numpy.int64'>'
	with 20507 stored elements in Compressed Sparse Row format>

In [22]:
# bulid model

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
nb_model=MultinomialNB()

In [25]:
nb_model.fit(x_train_dtmatrix,y_train) 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
y_prediction = nb_model.predict(x_test_dtmatrix) 

In [27]:
from sklearn import metrics

In [28]:
print(metrics.accuracy_score(y_test,y_prediction))

0.9808612440191388


In [29]:
print(metrics.confusion_matrix(y_test,y_prediction))

[[1431    8]
 [  24  209]]


In [30]:
# we bulid another model using logistial reg. model

In [31]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [32]:
log_model.fit(x_train_dtmatrix,y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
y_prediction2 = log_model.predict(x_test_dtmatrix) 

In [34]:
print(metrics.accuracy_score(y_test,y_prediction2))

0.9784688995215312


In [35]:
print(metrics.accuracy_score(y_test,y_prediction))

0.9808612440191388


In [36]:
# Multinomial test model is good

In [37]:
print(metrics.confusion_matrix(y_test,y_prediction2))

[[1433    6]
 [  30  203]]


In [38]:
print(metrics.confusion_matrix(y_test,y_prediction))

[[1431    8]
 [  24  209]]


### metrics.confusion

In [39]:
# 1431 is +ve(actual +ve) prediction is true ; 8 (fall +ve)  this actally +ve but prediction is falls
#  24 is (fall -ve) but prediction is falls ;209 is -ve(actual -ve) prediction is true
# finally we conclued that 1431 and 209  prediction are true
#                           24  and  8 prediction are falls

In [40]:
# y_test < y_prediction    ; 0,1

In [41]:
# 0<1 ham 0    ; spam 1
# fall +ve 
# achaly they are ham but predict​ as spam

In [42]:
# model one

In [43]:
x_test[y_test < y_prediction]

1672                              Glad to see your reply.
4622                   Received, understood n acted upon!
4862                               Nokia phone is lovly..
574                                Waiting for your call.
216     Finally the match heading towards draw as your...
991                                          26th OF JULY
4729    I (Career Tel) have added u as a contact on IN...
4702                               I liked the new mobile
Name: messages, dtype: object

In [44]:
len(x_test[y_test < y_prediction])

8

In [45]:
# y_test > y_prediction    ; 0,1

In [46]:
# 0>1 ham 0    ; spam 1
# fall -ve 
# achaly they are ham but predict​ as spam

In [47]:
x_test[y_test > y_prediction]

2558    This message is brought to you by GMW Ltd. and...
1500    SMS. ac JSco: Energy is high, but u may not kn...
2354    Please CALL 08712402902 immediately as there i...
4527    I want some cock! My hubby's away, I need a re...
3425    Am new 2 club & dont fink we met yet Will B gr...
4298    thesmszone.com lets you send free anonymous an...
3064    Hi babe its Jordan, how r u? Im home from abro...
3391    Please CALL 08712402972 immediately as there i...
731     Email AlertFrom: Jeri StewartSize: 2KBSubject:...
684     Hi I'm sue. I am 20 years old and work as a la...
4821    Check Out Choose Your Babe Videos @ sms.shsex....
4213    Missed call alert. These numbers called but le...
1663    Hi if ur lookin 4 saucy daytime fun wiv busty ...
751     Do you realize that in about 40 years, we'll h...
1940    More people are dogging in your area now. Call...
672     SMS. ac sun0819 posts HELLO:"You seem cool, wa...
1269    Can U get 2 phone NOW? I wanna chat 2 set up m...
4069    TBS/PE

In [48]:
len(x_test[y_test > y_prediction])

24

In [49]:
# model two

In [50]:
# fall +ve 

In [51]:
x_test[y_test < y_prediction2]

1384    Please reserve ticket on saturday eve from che...
5085    Sorry i missed your call. Can you please call ...
1866    Call to the number which is available in appoi...
3692    Sir, i am waiting for your call, once free ple...
4488    Miss call miss call khelate kintu opponenter m...
4729    I (Career Tel) have added u as a contact on IN...
Name: messages, dtype: object

In [52]:
# fall -ve 

In [53]:
x_test[y_test > y_prediction2]

2558    This message is brought to you by GMW Ltd. and...
1500    SMS. ac JSco: Energy is high, but u may not kn...
3750    Dear Voucher Holder 2 claim your 1st class air...
4616    Sunshine Quiz! Win a super Sony DVD recorder i...
1460    Bought one ringtone and now getting texts cost...
2269                    88066 FROM 88066 LOST 3POUND HELP
4047        Win a £1000 cash prize or a prize worth £5000
3425    Am new 2 club & dont fink we met yet Will B gr...
731     Email AlertFrom: Jeri StewartSize: 2KBSubject:...
684     Hi I'm sue. I am 20 years old and work as a la...
415     100 dating service cal;l 09064012103 box334sk38ch
4821    Check Out Choose Your Babe Videos @ sms.shsex....
4213    Missed call alert. These numbers called but le...
4914    Goal! Arsenal 4 (Henry, 7 v Liverpool 2 Henry ...
660     88800 and 89034 are premium phone services cal...
4506    Mobile Club: Choose any of the top quality ite...
1640    FreeMsg:Feelin kinda lnly hope u like 2 keep m...
751     Do you

In [54]:
len(x_test[y_test > y_prediction2])

30

### let find out the common spam and ham words

In [56]:
x_train_world = vector.get_feature_names()

In [57]:
len(x_train_world)

7125

In [59]:
x_train_world[7010:]


['wrld',
 'wrong',
 'wrongly',
 'wrote',
 'ws',
 'wt',
 'wtc',
 'wtf',
 'wud',
 'wudn',
 'wuld',
 'wuldnt',
 'wun',
 'www',
 'wylie',
 'x2',
 'x29',
 'x49',
 'xafter',
 'xam',
 'xavier',
 'xchat',
 'xclusive',
 'xin',
 'xmas',
 'xoxo',
 'xt',
 'xuhui',
 'xx',
 'xxuk',
 'xxx',
 'xxxmobilemovieclub',
 'xxxx',
 'xxxxx',
 'xxxxxx',
 'xxxxxxx',
 'xy',
 'y87',
 'ya',
 'yahoo',
 'yalrigu',
 'yam',
 'yan',
 'yar',
 'yarasu',
 'yavnt',
 'yaxx',
 'yaxxx',
 'yay',
 'yeah',
 'year',
 'years',
 'yeesh',
 'yeh',
 'yelling',
 'yellow',
 'yelow',
 'yen',
 'yeovil',
 'yep',
 'yer',
 'yes',
 'yest',
 'yesterday',
 'yet',
 'yetunde',
 'yifeng',
 'yijue',
 'ym',
 'ymca',
 'yo',
 'yoga',
 'yogasana',
 'yor',
 'yorge',
 'you',
 'youdoing',
 'youi',
 'young',
 'younger',
 'youphone',
 'your',
 'youre',
 'yourjob',
 'yours',
 'yourself',
 'youuuuu',
 'youwanna',
 'yowifes',
 'yoyyooo',
 'yr',
 'yrs',
 'ystrday',
 'ything',
 'yummmm',
 'yummy',
 'yun',
 'yunny',
 'yuou',
 'yup',
 'yupz',
 'zac',
 'zaher',
 'ze